In [1]:
import json

In [3]:
!ls ../multihop_dense_retrieval/data/hotpot/

dev_retrieval_b50_k50_sp.json  hotpot_test_fullwiki_v1.json
dev_retrieval_top100_sp.json   hotpot_train_firsthop.json
hotpot_dev_firsthop.json       hotpot_train_v1.1.json
hotpot_dev_fullwiki_v1.json    hotpot_train_with_neg_v0.json
hotpot_dev_with_neg_v0.json    train_retrieval_b100_k100_sp.json
hotpot_qas_val.json


In [4]:
SPLIT = "train"

In [5]:
# Read training data
fp_hotpot_mdr = '../multihop_dense_retrieval/data/hotpot/hotpot_train_with_neg_v0.json'
fp_hotpot_original = '../multihop_dense_retrieval/data/hotpot/hotpot_train_v1.1.json'

hotpot_mdr = []
with open(fp_hotpot_mdr, 'r') as handle:
    lines = handle.readlines()
    for l in lines:
        hotpot_mdr.append(json.loads(l))
    
with open(fp_hotpot_original, 'r') as handle:
    hotpot_orig = json.load(handle)

In [6]:
# Training data length
len(hotpot_orig)

90447

In [7]:
# Example training object
hotpot_orig[0]

{'supporting_facts': [["Arthur's Magazine", 0], ['First for Women', 0]],
 'level': 'medium',
 'question': "Which magazine was started first Arthur's Magazine or First for Women?",
 'context': [['Radio City (Indian radio station)',
   ["Radio City is India's first private FM radio station and was started on 3 July 2001.",
    ' It broadcasts on 91.1 (earlier 91.0 in most cities) megahertz from Mumbai (where it was started in 2004), Bengaluru (started first in 2001), Lucknow and New Delhi (since 2003).',
    ' It plays Hindi, English and regional songs.',
    ' It was launched in Hyderabad in March 2006, in Chennai on 7 July 2006 and in Visakhapatnam October 2007.',
    ' Radio City recently forayed into New Media in May 2008 with the launch of a music portal - PlanetRadiocity.com that offers music related news, videos, songs, and other music-related features.',
    ' The Radio station currently plays a mix of Hindi and Regional music.',
    ' Abraham Thomas is the CEO of the company.']]

In [8]:
# Count number of questions for which the context is not available for the SUP sentences

count_one, count_both, not_two = 0,0,0
for h in hotpot_orig:
    sup_set = set()
    for sup in h['supporting_facts']:
        sup_set.add(sup[0])
    context_set = set()
    for context in h['context']:
        context_set.add(context[0])
    if len(sup_set.intersection(context_set)) == 0:
        print(h)
        break
        count_both += 1
    if len(sup_set) <2:
        not_two += 1
print(count_both)
print(not_two)

0
0


In [9]:
# Check if the MDR-processed hotpot data is the same length
len(hotpot_mdr)

90447

In [10]:
# Example object in MDR hotpot
hotpot_mdr[0]

{'question': "Which magazine was started first Arthur's Magazine or First for Women?",
 'answers': ["Arthur's Magazine"],
 'type': 'comparison',
 'pos_paras': [{'title': "Arthur's Magazine",
   'text': 'Arthur\'s Magazine (1844–1846) was an American literary periodical published in Philadelphia in the 19th century. Edited by T.S. Arthur, it featured work by Edgar A. Poe, J.H. Ingraham, Sarah Josepha Hale, Thomas G. Spear, and others. In May 1846 it was merged into "Godey\'s Lady\'s Book".'},
  {'title': 'First for Women',
   'text': "First for Women is a woman's magazine published by Bauer Media Group in the USA. The magazine was started in 1989. It is based in Englewood Cliffs, New Jersey. In 2011 the circulation of the magazine was 1,310,696 copies."}],
 'neg_paras': [{'title': 'Knowledge (magazine)',
   'text': "Knowledge: An Illustrated Magazine of Science was a British popular science magazine published from 1881 to 1918. Founded by astronomer Richard A. Proctor, the magazine star

In [11]:
def extract_sentences(sup_idx_list, context_list, doc_title=None, return_titles=False):
    return_obj = []
    return_obj_titles = []
    sup = {}
    for s in sup_idx_list:
        if s[0] not in sup:
            sup[s[0]] = []
        sup[s[0]].append(s[1])
    for c in context_list:
        if c[0] not in sup:
            continue
        if doc_title is not None and c[0] != doc_title:
            continue
        for i in sup[c[0]]:
            try:
                return_obj.append(c[1][i])
                if return_titles:
                    return_obj_titles.append(c[0])
            except:
                print("IndexError: ignoring")
                print(c[0])
                print(c[1])
                print(sup[c[0]])
    if return_titles:
        return return_obj, return_obj_titles
    return return_obj

In [37]:
output = []

for i in range(len(hotpot_orig)):
    orig_obj = hotpot_orig[i]
    mdr_obj = hotpot_mdr[i]
    q_obj = {
        "id": orig_obj["_id"],
        "original_idx": i,
        "question": orig_obj["question"],
        "level": orig_obj["level"],
        "type": orig_obj["type"],
        "answers": [],
        "titles": [],
        "final_answer": orig_obj["answer"],
    }
    if orig_obj["type"] == "comparison":
        # Fetch all SUP sentences into "answers"
        q_obj["answers"], q_obj["titles"] = extract_sentences(orig_obj["supporting_facts"], 
                                                              orig_obj["context"], 
                                                              return_titles=True)
    else:
        # Fetch only the first-hop SUP sentences into "answers"
        for sup in orig_obj["supporting_facts"]:
            if sup[0] != mdr_obj["bridge"]:  # sup[0] -> doc title
                # Add intermediate answer sentences
                q_obj["answers"], q_obj["titles"] = extract_sentences(orig_obj["supporting_facts"], 
                                                                      orig_obj["context"], 
                                                                      sup[0], 
                                                                      return_titles=True)
                q_obj["final_title"] = mdr_obj["bridge"]
                break
    output.append(q_obj)

IndexError: ignoring
Minoru Suzuki
['Minoru Suzuki (鈴木 実 , Suzuki Minoru , ring name: 鈴木 みのる) (born June 17, 1968) is a Japanese professional wrestler and mixed martial artist who is currently working for New Japan Pro Wrestling (NJPW) as a freelancer.', ' He is the current NEVER Openweight Champion in his first reign.']
[0, 1, 2]
IndexError: ignoring
Khady Sylla
['Khady Sylla (Dakar, March 27, 1963 – Dakar, October 8, 2013) was a Senegalese writer of two novels, short work, and film.']
[0, 2]
IndexError: ignoring
Harry Styles
['Harry Edward Styles (born 1 February 1994) is an English singer, songwriter, and actor.', ' He made his debut as a singer with his band White Eskimo, who performed locally in Holmes Chapel, Cheshire.', ' Styles rose to stardom as a member of the boy band One Direction, formed in 2010 through the British music competition series "The X Factor".', ' One Direction has released five albums, performed four worldwide tours, and won several awards.']
[0, 3, 4]
IndexEr

In [38]:
output[1]

{'id': '5a879ab05542996e4f30887e',
 'original_idx': 1,
 'question': 'The Oberoi family is part of a hotel company that has a head office in what city?',
 'level': 'medium',
 'type': 'bridge',
 'answers': ['The Oberoi family is an Indian family that is famous for its involvement in hotels, namely through The Oberoi Group.'],
 'titles': ['Oberoi family'],
 'final_answer': 'Delhi',
 'final_title': 'The Oberoi Group'}

In [50]:
# Store results to disk
output_fp = f"../multihop_dense_retrieval/data/hotpot/hotpot_{SPLIT}_firsthop.json"
with open(output_fp, 'w') as fp:
    json.dump({"data": output}, fp, indent=4)

In [25]:
import numpy as np

In [41]:
np.random.seed(17)
shuffled_idxs = np.random.choice(len(output), len(output), replace=False)
train_idxs, dev_idxs = shuffled_idxs[:int(0.9*len(output))], shuffled_idxs[int(0.9*len(output)):]

In [47]:
assert (len(train_idxs) + len(dev_idxs)) == len(output)

In [48]:
train_split = list(np.array(output)[train_idxs])
dev_split = list(np.array(output)[dev_idxs])

In [49]:
train_split[0]

{'id': '5a88f50e55429946c8d6e8e2',
 'original_idx': 26522,
 'question': 'Claude Simon and Paul Valéry, are in which mutual industry?',
 'level': 'easy',
 'type': 'comparison',
 'answers': [' Valéry was nominated for the Nobel Prize in Literature in 12 different years.',
  'Claude Simon (] ; 10 October 1913 – 6 July 2005) was a French novelist and critic, and the 1985 Nobel Laureate in Literature.'],
 'titles': ['Paul Valéry', 'Claude Simon'],
 'final_answer': 'Literature'}

In [51]:
dev_split[0]

{'id': '5a7e06ed5542995f4f40239e',
 'original_idx': 44534,
 'question': 'Weer en Verkeer is a channel that provided weather forecasts from a town in what province?',
 'level': 'medium',
 'type': 'bridge',
 'answers': [' The forecasts and traffic reports were provided by a major Dutch meteorological company Meteo Consult in Wageningen which also provides the weather forecasts for RTL 4.'],
 'titles': ['Weer en Verkeer'],
 'final_answer': 'Gelderland',
 'final_title': 'Wageningen'}

In [52]:
# Store train split to disk
output_fp = f"../multihop_dense_retrieval/data/hotpot/hotpot_train_train_firsthop.json"
with open(output_fp, 'w') as fp:
    json.dump({"data": train_split}, fp, indent=4)
    
# Store dev split to disk
output_fp = f"../multihop_dense_retrieval/data/hotpot/hotpot_train_dev_firsthop.json"
with open(output_fp, 'w') as fp:
    json.dump({"data": dev_split}, fp, indent=4)